In [1]:
import re
import numpy as np
import pandas as pd
import math
import json
import collections
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#my_data = open("data200.json", "r")
# data = pd.read_json (r'C:\Flatiron\course-material\ModuleOneFinalProject\dsc-mod-1-project-v2-1-onl01-dtsc-pt-030220\data_0423_0749PM_30pages_2010_2020.json')
# df = df = pd.DataFrame(data)
# # df.info()
# print(df.shape)
# #df.describe()
# df[1465:1470]

int_columns = ['Budget','Gross USA', 'Cumulative Worldwide Gross', 'Popularity',
                'Opening Weekend USA', 'Opening Weekend USA','IMDB_Rate', 'Rating_count',
                'Meta_Score', 'User_Review_Number', 'Critical_Review_Number']

def load_data():
    data = pd.read_json (r'C:\Flatiron\course-material\ModuleOneFinalProject\dsc-mod-1-project-v2-1-onl01-dtsc-pt-030220\data_0423_0749PM_30pages_2010_2020.json')
    return pd.DataFrame(data)


In [3]:
# int_columns = ['Budget','Gross USA','Cumulative Worldwide Gross', 'Popularity',
#                'Opening Weekend USA','Opening Weekend USA',
#                'IMDB_Rate','Rating_count', 'Meta_Score','User_Review_Number',
#               'Critical_Review_Number']
# for column in int_columns:
#     df[column] = pd.to_numeric(df[column],errors='coerce')

def to_numeric(df):
    df[int_columns] = df[int_columns].apply(pd.to_numeric, errors='coerce')

    return df
    
# df = load_data()
# df = to_numeric(df)
# df.head()

In [4]:
# non_miss_Budegets = df['Budget'].loc[df['Budget'] > 0]
# non_miss_Budegets.isna().sum()
# display(non_miss_Budegets.apply(['mean', 'median', 'std']))
# non_miss_Budegets.hist()

def budget_hist(df):
    non_miss_Budegets = df['Budget'].loc[df['Budget'] > 0]
    non_miss_Budegets.isna().sum()
    display(non_miss_Budegets.apply(['mean', 'median', 'std']))
    non_miss_Budegets.hist()

In [5]:
# def creat_genre_df(col_name):

#     Genres = []
#     Title_id = []
#     money = [] 
#     for index, genres in enumerate(df['Genre']):
#          if(isinstance(genres, list)):
#             for genre in genres:
#                 Genres.append(genre)
#                 Title_id.append(df['Title_id'][index])
#                 money.append(df[col_name][index])
#     #            print(Genres, Title_id, Genres)

#     genres_dic = {'Title_id':Title_id, 'Genres': Genres}
#     genres_dic[col_name] = money
#     genres_df = pd.DataFrame(genres_dic)
#     return genres_df

# creat_genre_df('Budget')



# https://www.mikulskibartosz.name/how-to-split-a-list-inside-a-dataframe-cell-into-rows-in-pandas/
def flatten_genre(df):
    flatten_df = df['Genre'].apply(pd.Series) \
        .merge(df, right_index = True, left_index = True) \
        .drop(['Genre'], axis = 1) \
        .melt(id_vars = df.columns.drop('Genre'), value_name = "Genre") \
        .drop("variable", axis = 1) \
        .dropna(subset=['Genre'])
    return flatten_df
#     print(df2.isna().sum())

# flatten_genre(df)

In [6]:
# All_Genres = []
# gener_mean = {}
# for index, genres in enumerate(df['Genre']):
#          if(isinstance(genres, list)):
#             for genre in genres:
#                 All_Genres.append(genre)
# unique_genres = set(All_Genres)
# unique_genres
# for genre in unique_genres:
#     gener_mean[genre] = creat_genre_df('Budget').loc[creat_genre_df('Budget')['Genres']== genre].mean()


# df must be the flatten df
# genre column must not be list
def get_all_genres_count(flatten_df):
    return flatten_df.groupby('Genre')['Genre'].count()



# df = load_data()
# df = to_numeric(df)
# flatten_df = flatten_genre(df)
# all_genres = get_all_genres_count(flatten_df)
# # print(len(all_genres))
# all_genres['Action']


In [7]:
# unique_genres = set(All_Genres)

# total_rows =len(All_Genres)
# genre_country_list = collections.Counter(All_Genres)


# def one_genre_weight(movie_genre):
#     All_Genres = []
#     for index, genres in enumerate(df['Genre']):
#              if(isinstance(genres, list)):
#                 for genre in genres:
#                     All_Genres.append(genre)
#     #print(All_Genres)
#     genre_country = collections.Counter(All_Genres)
#     weight = genre_country[movie_genre]/len(All_Genres)
#     #print(weight)
#     return weight
# one_genre_weight('War')



In [8]:
# def all_genres_weight (genre_country_list):
#     genres_weight = {}
#     for genre in genre_country_list:
#         genres_weight[genre] = one_genre_weight(genre)
#     return genres_weight
# all_genres_weight (genre_country_list)


def get_genres_weight(all_genres_count, flatten_df):
    return all_genres_count / len(flatten_df)



# df = load_data()
# df = to_numeric(df)
# flatten_df = flatten_genre(df)
# all_genres_count = get_all_genres_count(flatten_df)
# genres_weight = get_genres_weight(all_genres_count, flatten_df)
# genres_weight.head()
# # print(all_genres_count / 10)
# # print(len(all_genres))
# # all_genres['Action']

In [9]:
def remove_outlier(df, col_name):
    q1 = df[col_name].quantile(0.25)
    q3 = df[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df.loc[(df[col_name] > fence_low) & (df[col_name] < fence_high)]
    return df_out


In [10]:
# def get_per_header_stat_df(df_name, column, group_by_column):
#     new_df = remove_outlier(df_name, column)
#     no_outlier_new_df = new_df.loc[new_df[column] > 0 ].groupby(group_by_column).agg({column: ['mean', 'median']})
#     return no_outlier_new_df
# get_per_header_stat_df(creat_genre_df('Budget'),'Budget', 'Genres')


def get_per_column_stat(flatten_df, column, group_by_column):
    no_outlier_df = remove_outlier(flatten_df, column)
    return no_outlier_df.groupby(group_by_column).agg({column: ['mean', 'median']})



# df = load_data()
# df = to_numeric(df)
# flatten_df = flatten_genre(df)
# all_genres_count = get_all_genres_count(flatten_df)
# genres_weight = get_genres_weight(all_genres_count, flatten_df)
# per_column_stat = get_per_column_stat(flatten_df, 'Budget', 'Genre')
# per_column_stat.head()
# # flatten_df.head()
# # genres_weight.head()
# # print(all_genres_count / 10)
# # print(len(all_genres))
# # all_genres['Action']

In [11]:
# genre_budget_df = creat_genre_df('Budget')
# header_stat = get_per_header_stat_df(genre_budget_df,'Budget', 'Genres')
# a = 0
# calculated_all_genres_weight = all_genres_weight(genre_country_list)
# for index, genres in enumerate(df['Genre']):
#     if math.isnan(df['Budget'][index]) :
#         a += 1
#         weight_sum = 0
#         weighted_mean = 0
#         for movie_genre in genres:
#             weight_sum = weight_sum +calculated_all_genres_weight[movie_genre]
#             weighted_mean = weighted_mean + (header_stat['Budget']['mean'][movie_genre] *calculated_all_genres_weight[movie_genre])

#         df.iloc[index, df.columns.get_loc('Budget')] = weighted_mean / weight_sum

# df.isna().sum()           


def estimate_missing_column(df, per_column_stat, genres_weight, column_name):
    for index, genres in enumerate(df['Genre']):
        if math.isnan(df[column_name][index]) :
            weight_sum = 0
            weighted_mean = 0
            for movie_genre in genres:
                if (movie_genre in per_column_stat[column_name]['mean']):
                    weight_sum += genres_weight[movie_genre]
                    weighted_mean += \
                            per_column_stat[column_name]['mean'][movie_genre] * \
                            genres_weight[movie_genre]
            df.iloc[index, df.columns.get_loc(column_name)] = weighted_mean / weight_sum
    return df

def get_ready_to_analyze_data():
    df = load_data()
    df = to_numeric(df)
    flatten_df = flatten_genre(df)
    all_genres_count = get_all_genres_count(flatten_df)
    genres_weight = get_genres_weight(all_genres_count, flatten_df)
    for column in ['Budget', 'Opening Weekend USA', 'Gross USA', 'Cumulative Worldwide Gross']:
        per_column_stat = get_per_column_stat(flatten_df, column, 'Genre')
        df = estimate_missing_column(df, per_column_stat, genres_weight, column)
    return df

ready_to_analyze_data = get_ready_to_analyze_data()
ready_to_analyze_data.isna().sum()
# no_missing_df.head()


Title_id                        0
Budget                          0
Gross USA                       0
Cumulative Worldwide Gross      0
Opening Weekend USA             0
Title                           7
Year                            0
Popularity                      0
Genre                           0
Runtime_min                    33
Release_day                     5
IMDB_Rate                       1
Rating_count                    1
Meta_Score                    895
Director                        0
Writer                          0
Star                            0
User_Review_Number              7
Critical_Review_Number          7
Meta_Rating                     0
dtype: int64

In [12]:
#creat_genre_df('Budget').loc[creat_genre_df('Budget')['Genres']=='Action'].mean()

In [13]:
# data_genres_df = pd.melt(genres_df, id_vars=['Genres'],value_vars=['Budget'],var_name='Gross')
# sns.boxplot(x='Genres',y='value',data=data_genres_df,hue='Gross')

In [14]:
#genres_df.groupby('Genres').mean()

In [15]:
# error_list = []
# for i, item in enumerate(df['Budget:']):
#     try:
#         int(item)
#     except ValueError:
#         #error_list.append('ERROR at index {}: {!r}'.format(i, item))
#         error_list.append(i)
# print(error_list)
# len(error_list)
#df = df.drop(error_list, axis=0)

In [16]:
# curencys = [re.split(r'(\d+)', error) for error in error_list]
# print(curencys)
# curency_used = []
# for curency in curencys:
#     curency_used.append(curency[0])
# uniqe_currency = list(set(curency_used))
# uniqe_currency

In [17]:
# df = df.replace({'None': None})
# df.head()
# count = 0
# for i, item in enumerate(df['Budget:']):
#     if item == 'None':
# #     if item is None:
#         count += 1
# print(count)

In [18]:
# from scipy import stats
# z_scores = stats.zscore(genres_df)
# abs_z_scores = np.abs(z_scores)
# filtered_entries = (abs_z_scores < 3).all(axis=1)
# new_df = df[filtered_entries]